BELOW IS THE PART OF THE CODE THAT IS NECESSARY TO RUN IN ORDER TO DOWNGRADE CUDA TO VERSION 9.2, THE NEWEST VERSION 11 DOESN'T WORK WITH THE NVCC PLUGIN FOR NOTEBOOKS. THIS IS ONLY EXECUTED IN THIS SPECIFIC NOTEBOOK SO THERE IS NO DANGER OF DOWNGRADNG ANYTHING IN THE LOCAL MACHINE. 

IF THE NOTEBOOK IS RESTARTED THEN THE VERSION GOES BACK TO 11.

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

AFTER UNINSTALLING CUDA THE CODE BELOW REINSTALLS THE OLDER VERSION 9.2 THAT CAN WORK WITH THE NVCC PLUGIN.

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

LOADING THE NVCC PLUGING.

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%reload_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-n425rv3k
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-n425rv3k
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9487f8a61631c0af4105f94bf83f904afd5185b392ca830b65140084024d2fd9
  Stored in directory: /tmp/pip-ephem-wheel-cache-za_nwir4/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


HELLO WORLD CODE JUST TO CHECK IT IS WORKING.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>

__global__ void print_from_gpu(void) {
    printf("Hello World! from thread [%d,%d]", threadIdx.x,blockIdx.x);
}

int main(void) {

    printf("Hello World from host!");
    print_from_gpu<<<2,1>>>();
    cudaDeviceSynchronize();
    return 0;
}

Hello World from host!Hello World! from thread [0,0]Hello World! from thread [0,1]


FIRST CODE USING A FOR LOOP TO CALCULATE ONE MATRIX MULTIPLICATION AFTER THE OTHER.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
__global__ void matrixmul(int *A, int* B, int* C, int dim_n, int dim_k){
    //find row and column
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int column = blockIdx.x * blockDim.x + threadIdx.x;
    int p_sum = 0;
    for (int i = 0; i < dim_n; i++){
        p_sum += A[row * dim_n + i] * B[i * dim_k+ column];
    }
    C[row * dim_k + column] = p_sum;
}

void arrPrintMatrix(int *matrix, int m, int x) {
    int i, j;

    for (i = 0; i < m; i++) {
        printf("\n");
        for (j = 0; j < x; j++) {
            printf("%d\t", matrix[i * x + j]);
        }
    }
    printf("\n");
}


int main() {
    
    clock_t begin = clock();
    // THIS FOR LOOP IS FOR CALCULATING Cj ONE AT A TIME.
    for (int N = 0; N < 100; N++){
    // I CHOSE THE EXAMPLE IN WHICH THE MATRICES ARE A = 500x500 AND B = 500x400.    
    int n = 500;
    int k = 400;

    // host matrix m,n,p
    int* h_m;
    int* h_n;
    int* h_p;

    //device matrix m,n,p
    int* d_m;
    int* d_n;
    int* d_p;

    size_t bytes = n*n*sizeof(int);

    //allocate memory on host side

    h_m = (int*)malloc(bytes);
    h_n = (int*)malloc(bytes);
    h_p = (int*)malloc(bytes);

    //initialize matrix A
    for (int i = 0; i < n; i++){
        for (int j = 0; j < n; j++){
            h_m[i*n + j] = rand() % 1024;

        }
    }

    //initialize matrix B
    for (int i = 0; i < n; i++){
        for (int j = 0; j < k; j++){
          
            h_n[i*k + j] = rand() % 1024;

        }
    }

    //arrPrintMatrix(h_m,n,n);
    //arrPrintMatrix(h_n,n,k);

    //allocate memory gpu
    cudaMalloc(&d_m, bytes);
    cudaMalloc(&d_n, bytes);
    cudaMalloc(&d_p, bytes);

    cudaMemcpy(d_m, h_m, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_n, h_n, bytes, cudaMemcpyHostToDevice);


    int threads_per_block = 128;
    dim3 block_size(threads_per_block, threads_per_block);
    dim3 grid_size(n/block_size.x, n/block_size.y);
    matrixmul <<<grid_size, block_size>>>(d_m, d_n, d_p, n,k);

    cudaMemcpy(h_p, d_p, bytes, cudaMemcpyDeviceToHost);
    
    //arrPrintMatrix(h_p, n,k);
    }
    clock_t end = clock();
    double time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    printf ("Your calculations took %.2lf seconds to run.\n", time_spent );
    printf("---------------------------==========--------------------------");
    return 0;
}

Your calculations took 1.32 seconds to run.
---------------------------==========--------------------------


THIS CODE BELOW IS THE EXAMPLE IN WHICH MATRICES A AND B ARE 4x4 and 4x3, N IS 10. THIS IS TO SHOW THE CODE IS WORKING AND TO PRINT SOME EXAMPLES.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<time.h>
__global__ void matrixmul(int *A, int* B, int* C, int dim_n, int dim_k){
    //find row and column
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int column = blockIdx.x * blockDim.x + threadIdx.x;
    int p_sum = 0;

    if( (row < dim_n) && (column < dim_k)){
    for (int i = 0; i < dim_n; i++){
        p_sum += A[row * dim_n + i] * B[i * dim_k+ column];
    }
    C[row * dim_k + column] = p_sum;
    }
}

void arrPrintMatrix(int *matrix, int m, int x) {
    int i, j;

    for (i = 0; i < m; i++) {
        printf("\n");
        for (j = 0; j < x; j++) {
            printf("%d\t", matrix[i * x + j]);
        }
    }
    printf("\n");
}


int main() {
    
    clock_t begin = clock();
    // THIS FOR LOOP IS FOR CALCULATING Cj ONE AT A TIME.
    for (int N = 0; N < 10; N++){
    // I CHOSE THE EXAMPLE IN WHICH THE MATRICES ARE A = 5x5 AND B = 5x4.    
    int n = 5;
    int k = 4;

    // host matrix m,n,p
    int* h_m;
    int* h_n;
    int* h_p;

    //device matrix m,n,p
    int* d_m;
    int* d_n;
    int* d_p;

    size_t bytes = n*n*sizeof(int);

    //allocate memory on host side

    h_m = (int*)malloc(bytes);
    h_n = (int*)malloc(bytes);
    h_p = (int*)malloc(bytes);

    //initialize matrix A
    for (int i = 0; i < n; i++){
        for (int j = 0; j < n; j++){
            h_m[i*n + j] = rand() % 1024;

        }
    }

    //initialize matrix B
    for (int i = 0; i < n; i++){
        for (int j = 0; j < k; j++){
          
            h_n[i*k + j] = rand() % 1024;

        }
    }

    arrPrintMatrix(h_m,n,n);
    arrPrintMatrix(h_n,n,k);

    //allocate memory gpu
    cudaMalloc(&d_m, bytes);
    cudaMalloc(&d_n, bytes);
    cudaMalloc(&d_p, bytes);

    cudaMemcpy(d_m, h_m, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_n, h_n, bytes, cudaMemcpyHostToDevice);


    int threads_per_block = 8;
    dim3 block_size(threads_per_block, threads_per_block);
    //dim3 grid_size(n/block_size.x, n/block_size.y);
    dim3 grid_size(1,1);
    matrixmul <<<grid_size, block_size>>>(d_m, d_n, d_p, n,k);

    cudaMemcpy(h_p, d_p, bytes, cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();
    cudaFree(d_m);
    cudaFree(d_n);
    cudaFree(d_p);
    arrPrintMatrix(h_p, n,k);
    }
    clock_t end = clock();
    double time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    printf ("Your calculations took %.2lf seconds to run.\n", time_spent );
    printf("---------------------------==========--------------------------");
    return 0;
}


359	966	105	115	81	
255	74	236	809	205	
186	939	498	763	483	
326	124	706	84	1016	
795	488	487	909	886	

346	302	611	563	
927	201	922	870	
306	13	951	561	
88	163	346	293	
349	261	791	88	

1090215	343835	1313717	1142265	
371781	280324	890538	595418	
1322908	501817	2099053	1467089	
805756	411422	1817640	801504	
1265674	723922	2414158	1489657	

745	94	212	427	178	
205	198	667	692	84	
529	14	386	116	577	
289	317	476	135	624	
489	62	161	577	225	

508	871	574	769	
638	663	490	732	
875	918	911	56	
92	554	748	176	
59	763	562	175	

673718	1278205	1086254	759887	
882709	1369595	1387151	476425	
660129	1328904	1073194	560056	
814794	1449760	1206520	613901	
495202	1106156	1015783	571368	

316	852	492	792	987	
92	258	26	254	835	
251	762	682	826	507	
297	465	998	5	316	
892	916	373	984	446	

97	137	505	860	
699	680	153	527	
149	945	491	241	
179	517	495	1015	
768	233	673	570	

1599292	1727027	1587799	2205806	
879886	538487	786385	955112	
1195833	1742210	1328284	1909176	
746129	1376212	926291	926188	
13

THE CODE BELOW EXECUTES THE SAME CALCULATIONS ON CUDA BUT THIS TIME USING 3D BLOCKS AND THREADS. MATRIX A IS 500x500x100 AND MATRIX B = 500x400x100.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<time.h>

__global__ void matrixmul(int *A, int* B, int* C, int dim_n, int dim_k, int quantity){
    

    //find row and column and the index of the matrix
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int column = blockIdx.x * blockDim.x + threadIdx.x;
    int matrix_index = blockIdx.z*blockDim.z + threadIdx.z;


    if( (row < dim_n) && (column < dim_k) && (matrix_index < quantity)){
        
        int p_sum = 0;
        for (int i = 0; i < dim_n; i++){
            // 
            p_sum += A[matrix_index * (dim_n*dim_n) + row * dim_n + i] * B[matrix_index * (dim_n*dim_k) + i * dim_k+ column];
        }

        __syncthreads();
        C[matrix_index * (dim_n*dim_k) + row * dim_k + column] = p_sum;
    }

}

void arrPrintMatrix(int *matrix, int m, int x, int quantity) {
    
    int i, j;

    for (int q = 0; q < quantity; q++){
        
        for (i = 0; i < m; i++) {
            
            printf("\n");
            for (j = 0; j < x; j++) {
                
                printf("%d\t", matrix[q*(m*x) + i * x + j]);

            }

        }

    printf("\n");

    }

    printf("------------------------------------------\n\n");

}


int main() {
    
    clock_t begin = clock();
    int quantity = 100;
    int n = 500;
    int k = 400;

    // host matrix m,n,p
    int* h_m;
    int* h_n;
    int* h_p;

    //device matrix m,n,p
    int* d_m;
    int* d_n;
    int* d_p;

    size_t bytes = n*n*quantity*sizeof(int);

    //allocate memory on host side

    h_m = (int*)malloc(bytes);
    h_n = (int*)malloc(bytes);
    h_p = (int*)malloc(bytes);

    //initialize matrix A
    for (int q = 0; q < quantity; q ++){
        
        for (int i = 0; i < n; i++){
        for (int j = 0; j < n; j++){
            h_m[q*(n*n) + i*n + j] = rand() % 1024;

        }
    }
    }
    

    //initialize matrix B
    for (int q = 0; q < quantity; q ++){
        for (int i = 0; i < n; i++){
        for (int j = 0; j < k; j++){
          
            h_n[q*(n*k) + i*k + j] = rand() % 1024;

        }
    }
    }
  

    //arrPrintMatrix(h_m,n,n,quantity);
    //arrPrintMatrix(h_n,n,k,quantity);

    //allocate memory gpu
    cudaMalloc(&d_m, bytes);
    cudaMalloc(&d_n, bytes);
    cudaMalloc(&d_p, bytes);

    cudaMemcpy(d_m, h_m, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_n, h_n, bytes, cudaMemcpyHostToDevice);


    int threads_per_block = 128;
    dim3 block_size(threads_per_block, threads_per_block, threads_per_block);
    dim3 grid_size(n/block_size.x, n/block_size.y, quantity/block_size.z);
    matrixmul <<<grid_size, block_size>>>(d_m, d_n, d_p, n,k, quantity);
    cudaDeviceSynchronize();
    cudaMemcpy(h_p, d_p, bytes, cudaMemcpyDeviceToHost);
    //arrPrintMatrix(h_p, n,k, quantity);
    clock_t end = clock();
    double time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    printf ("Your calculations took %.2lf seconds to run.\n", time_spent );
    return 0;
}

Your calculations took 0.89 seconds to run.



THE EXAMPLE BELOW IS TO PROVE THAT USING THE THREE DIMENSIONS OF THE THREADS AND BLOCKS IS WORKING. I USED IN THIS CASE N = 5 , A is 5x5, AND B IS 5x4.

In [ ]:
%%cu
#include<stdio.h>
#include<stdlib.h>
#include<time.h>

__global__ void matrixmul(int *A, int* B, int* C, int dim_n, int dim_k, int quantity){
    

    //find row and column and the index of the matrix
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int column = blockIdx.x * blockDim.x + threadIdx.x;
    int matrix_index = blockIdx.z*blockDim.z + threadIdx.z;


    if( (row < dim_n) && (column < dim_k) && (matrix_index < quantity)){
        
        int p_sum = 0;
        for (int i = 0; i < dim_n; i++){
            // 
            p_sum += A[matrix_index * (dim_n*dim_n) + row * dim_n + i] * B[matrix_index * (dim_n*dim_k) + i * dim_k+ column];
        }

        __syncthreads();
        C[matrix_index * (dim_n*dim_k) + row * dim_k + column] = p_sum;
    }

}

void arrPrintMatrix(int *matrix, int m, int x, int quantity) {
    
    int i, j;

    for (int q = 0; q < quantity; q++){
        
        for (i = 0; i < m; i++) {
            
            printf("\n");
            for (j = 0; j < x; j++) {
                
                printf("%d\t", matrix[q*(m*x) + i * x + j]);

            }

        }

    printf("\n");

    }

    printf("------------------------------------------\n\n");

}


int main() {
    
    clock_t begin = clock();
    int quantity = 5;
    int n = 5;
    int k = 4;

    // host matrix m,n,p
    int* h_m;
    int* h_n;
    int* h_p;

    //device matrix m,n,p
    int* d_m;
    int* d_n;
    int* d_p;

    size_t bytes = n*n*quantity*sizeof(int);

    //allocate memory on host side

    h_m = (int*)malloc(bytes);
    h_n = (int*)malloc(bytes);
    h_p = (int*)malloc(bytes);

    //initialize matrix A
    for (int q = 0; q < quantity; q ++){
        
        for (int i = 0; i < n; i++){
        for (int j = 0; j < n; j++){
            h_m[q*(n*n) + i*n + j] = rand() % 1024;

        }
    }
    }
    

    //initialize matrix B
    for (int q = 0; q < quantity; q ++){
        for (int i = 0; i < n; i++){
        for (int j = 0; j < k; j++){
          
            h_n[q*(n*k) + i*k + j] = rand() % 1024;

        }
    }
    }
  

    arrPrintMatrix(h_m,n,n,quantity);
    arrPrintMatrix(h_n,n,k,quantity);

    //allocate memory gpu
    cudaMalloc(&d_m, bytes);
    cudaMalloc(&d_n, bytes);
    cudaMalloc(&d_p, bytes);

    cudaMemcpy(d_m, h_m, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy(d_n, h_n, bytes, cudaMemcpyHostToDevice);


    int threads_per_block = 1;
    dim3 block_size(threads_per_block, threads_per_block, threads_per_block);
    dim3 grid_size(n/block_size.x, n/block_size.y, quantity/block_size.z);
    matrixmul <<<grid_size, block_size>>>(d_m, d_n, d_p, n,k, quantity);
    cudaDeviceSynchronize();
    cudaMemcpy(h_p, d_p, bytes, cudaMemcpyDeviceToHost);
    arrPrintMatrix(h_p, n,k, quantity);
    clock_t end = clock();
    double time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    printf ("Your calculations took %.2lf seconds to run.\n", time_spent );
    return 0;
}


359	966	105	115	81	
255	74	236	809	205	
186	939	498	763	483	
326	124	706	84	1016	
795	488	487	909	886	

346	302	611	563	927	
201	922	870	306	13	
951	561	88	163	346	
293	349	261	791	88	
745	94	212	427	178	

205	198	667	692	84	
529	14	386	116	577	
289	317	476	135	624	
489	62	161	577	225	
508	871	574	769	638	

663	490	732	875	918	
911	56	92	554	748	
176	59	763	562	175	
316	852	492	792	987	
92	258	26	254	835	

251	762	682	826	507	
297	465	998	5	316	
892	916	373	984	446	
97	137	505	860	699	
680	153	527	149	945	
------------------------------------------


491	241	179	517	
495	1015	768	233	
673	570	741	970	
11	715	976	328	
583	868	701	543	

291	798	680	796	
635	356	453	788	
883	602	709	350	
843	889	867	315	
880	612	548	529	

158	265	476	170	
980	428	498	539	
272	175	59	563	
973	739	336	584	
71	789	348	955	

367	34	281	186	
923	125	501	779	
737	26	284	895	
291	760	41	248	
164	539	787	437	

714	846	1000	664	
562	312	224	633	
77	573	564	444	
607	846	631	506	
971	108	261	684	
------------------

THE PYTHON EQUIVALENT CODE THAT USES A FOR LOOP TO CALCULATE ONE MATRIX MULTIPLICATION AFTER THE OTHER.

In [ ]:
import numpy as np
import time

###### GENERATING RANDOM MATRICES #######
start = time.time()
A = np.random.randint(0,1024,(500,500,100))
B = np.random.randint(0,1024,(500,400,100))
C = []

for n in range(100):
  matrix_a = A[:,:,n]
  matrix_b = B[:,:,n]
  result = np.dot(matrix_a, matrix_b)
  C.append(result)

end = time.time()
print(f"The Python equivalent code took: {end - start} seconds to run.")
# print(end - start)


The Python equivalent code took: 33.971593141555786 seconds to run.
